## We extract only TXO and TX from 2020/03/13 to 2020/04/24

* Data resorce:    
    * Futures:  http://www.taifex.com.tw/cht/3/dlFutPrevious30DaysSalesData  
    * Options:  http://www.taifex.com.tw/cht/3/dlOptPrevious30DaysSalesData
* Data exclude:
    * Options: Only include the open 5 mins trading information, which is, 84500 ~ 85000
    * Futures: Include every transaction data

In [1]:
import pandas as pd 
import numpy as np 
import os, glob
import datetime as dt

In [2]:
work_dir = os.getcwd()
work_dir

'C:\\Users\\Evan\\Documents\\GitHub\\TXO_TX_Strategy'

In [3]:
Option_path = os.path.join(work_dir, 'Data', 'Option')
Future_path = os.path.join(work_dir, 'Data', 'Futures')
Future_path

'C:\\Users\\Evan\\Documents\\GitHub\\TXO_TX_Strategy\\Data\\Futures'

In [4]:
vacations = [dt.date(2020, 4, 2), dt.date(2020, 4, 3)] # Taiwan vacations

In [5]:
dates = pd.bdate_range(start='3/13/2020', end='4/24/2020')
dates = dates.drop(vacations)

In [6]:
dates.shape

(29,)

## Collect all the csv name

In [4]:
extension = 'csv'
os.chdir(Option_path)
Opt_csv_List = glob.glob('*.{}'.format(extension))

## Read the Option csv and data cleaning

In [96]:
def output_csv(dataCsv):
    # The product name needs to be careful with space
    df = pd.read_csv(os.path.join(Option_path, dataCsv), encoding = 'cp950')
    columns_name = ['TransDate', 'Product', 'ExcercisePrice', 'Maturity', 'Right', 'TransTime', 'TransPrice', 'Volumn', 'E']
    df.columns  = columns_name 
    df_new = df[(df.Product == '    TXO     ') & ((df.TransTime.between(left=84500, right=85000)) | (df.TransTime.between(left=134000, right=134500)))]
    df_new = df_new.drop('E', axis = 1)
    df_new.to_csv(os.path.join(Option_path, dataCsv), index = False)
    print("writing file to {}".format(dataCsv))

In [98]:
for dataCsv in Opt_csv_List: 
    output_csv(dataCsv)

C:\Users\Evan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


writing file to OptionsDaily_2020_03_16.csv
writing file to OptionsDaily_2020_03_17.csv


C:\Users\Evan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (0,3) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


writing file to OptionsDaily_2020_03_18.csv
writing file to OptionsDaily_2020_03_19.csv
writing file to OptionsDaily_2020_03_20.csv
writing file to OptionsDaily_2020_03_23.csv
writing file to OptionsDaily_2020_03_24.csv
writing file to OptionsDaily_2020_03_25.csv
writing file to OptionsDaily_2020_03_26.csv
writing file to OptionsDaily_2020_03_27.csv
writing file to OptionsDaily_2020_03_30.csv
writing file to OptionsDaily_2020_03_31.csv
writing file to OptionsDaily_2020_04_01.csv
writing file to OptionsDaily_2020_04_06.csv
writing file to OptionsDaily_2020_04_07.csv
writing file to OptionsDaily_2020_04_08.csv
writing file to OptionsDaily_2020_04_09.csv
writing file to OptionsDaily_2020_04_10.csv
writing file to OptionsDaily_2020_04_13.csv
writing file to OptionsDaily_2020_04_14.csv
writing file to OptionsDaily_2020_04_15.csv
writing file to OptionsDaily_2020_04_16.csv
writing file to OptionsDaily_2020_04_17.csv
writing file to OptionsDaily_2020_04_20.csv
writing file to OptionsDaily_202

## Read the Future csv and data cleaning

In [20]:
extension = 'csv'
os.chdir(Future_path)
Fut_csv_List = glob.glob('*.{}'.format(extension))
Future_path = os.path.join(work_dir, 'Data', 'Futures')

In [13]:
def output_csv2(dataCsv):
    df = pd.read_csv(os.path.join(Future_path, dataCsv), encoding = 'cp950')
    columns_name = ['TransDate', 'Product', 'Maturity', 'TransTime', 'TransPrice', 'Volumn', 'A', 'B', 'E']
    df.columns  = columns_name 
    df_new = df[(df.Product == 'TX     ') & (df.TransTime.between(left=84500, right=134500))] # The product name needs to be careful with space
    df_new = df_new.drop(['A', 'B', 'E'], axis = 1)
    df_new.to_csv(os.path.join(New_Future_path, dataCsv), index = False)
    print("writing file to {}".format(dataCsv))

In [14]:
for dataCsv in Fut_csv_List: 
    output_csv2(dataCsv)

ValueError: Length mismatch: Expected axis has 6 elements, new values have 9 elements

## Make sure the new csv is correct 

In [86]:
df = pd.read_csv(os.path.join(Option_path, Opt_csv_List[7]), encoding = 'cp950')
df.head()

,TransDate,Product,ExcercisePrice,Maturity,Right,TransTime,TransPrice,Volumn
0,20200324,TXO,7300.0,202004,P,84500.0,70.0,2.0
1,20200324,TXO,7300.0,202004,P,84500.0,70.0,1.0
2,20200324,TXO,7300.0,202004,P,84500.0,70.0,1.0
3,20200324,TXO,7300.0,202004,P,84500.0,70.0,1.0
4,20200324,TXO,7300.0,202004,P,84500.0,70.0,1.0


In [91]:
type(df.ExcercisePrice[0])

numpy.float64

In [77]:
df = pd.read_csv(os.path.join(Future_path, Fut_csv_List[0]), encoding = 'cp950')
len(df)

17481

## Slim down from all info to the vital info (Futures)

* For Futures, Only keep   
    * the nearby month contract  
    * There are multiple transaction within 1 sec, we use groupby to acquire the mean of the transaction Price as the representitive price


In [21]:
def futures_keep_info(dataCsv): 
    'Slim the Size'
    df = pd.read_csv(os.path.join(Future_path, dataCsv), encoding = 'cp950')
    meanPrice = round(df.groupby('TransTime').TransPrice.transform('mean'), 2)
    df['meanPrice'] = meanPrice
    nearbyMonth = df.Maturity[0]
    df_new = df[df.Maturity == nearbyMonth]

    df_new = df_new.drop_duplicates(subset = ['TransTime'], keep ='first')
    df_new.to_csv(os.path.join(Future_path, dataCsv), index = False)
    print("writing file to {}".format(dataCsv))

In [23]:
for dataCsv in Fut_csv_List: 
    futures_keep_info(dataCsv)

writing file to Daily_2020_03_13.csv
writing file to Daily_2020_03_16.csv
writing file to Daily_2020_03_17.csv
writing file to Daily_2020_03_18.csv
writing file to Daily_2020_03_19.csv
writing file to Daily_2020_03_20.csv
writing file to Daily_2020_03_23.csv
writing file to Daily_2020_03_24.csv
writing file to Daily_2020_03_25.csv
writing file to Daily_2020_03_26.csv
writing file to Daily_2020_03_27.csv
writing file to Daily_2020_03_30.csv
writing file to Daily_2020_03_31.csv
writing file to Daily_2020_04_01.csv
writing file to Daily_2020_04_06.csv
writing file to Daily_2020_04_07.csv
writing file to Daily_2020_04_08.csv
writing file to Daily_2020_04_09.csv
writing file to Daily_2020_04_10.csv
writing file to Daily_2020_04_13.csv
writing file to Daily_2020_04_14.csv
writing file to Daily_2020_04_15.csv
writing file to Daily_2020_04_16.csv
writing file to Daily_2020_04_17.csv
writing file to Daily_2020_04_20.csv
writing file to Daily_2020_04_21.csv
writing file to Daily_2020_04_22.csv
w

In [24]:
df = pd.read_csv(os.path.join(Future_path, Fut_csv_List[2]), encoding = 'cp950')
df.head()

,TransDate,Product,Maturity,TransTime,TransPrice,Volumn
0,20200317,TX,202003,84500,9379.0,1428
1,20200317,TX,202003,84500,9379.0,2
2,20200317,TX,202003,84500,9379.0,2
3,20200317,TX,202003,84500,9379.0,2
4,20200317,TX,202003,84500,9379.0,2


In [25]:
df.groupby('Maturity').Volumn.sum()

Maturity
202003           488126
202003/202004     50088
202003/202005         4
202003/202009        16
202003/202012         4
202004           105886
202004/202005       372
202004/202006        36
202005              792
202006              734
202009              198
202012              216
Name: Volumn, dtype: int64

In [81]:
type(df.TransDate[0])

numpy.int64

## Divided the Option data into Open interval and close interval 

In [5]:
OptionOpen_path = os.path.join(work_dir, 'Data', 'OptionOpen')
OptionClose_path = os.path.join(work_dir, 'Data', 'OptionClose')

In [6]:
def divide_csv(dataCsv):
    df = pd.read_csv(os.path.join(Option_path, dataCsv), encoding = 'cp950')
    df_open = df[(df.TransTime.between(left=84500, right=85000))] 
    df_close = df[(df.TransTime.between(left=134000, right=134500))] 
    df_open.to_csv(os.path.join(OptionOpen_path, dataCsv), index = False)
    df_close.to_csv(os.path.join(OptionClose_path, dataCsv), index = False)
    print("writing file to {}".format(dataCsv))

In [8]:
OptionOpen_path

'C:\\Users\\Evan\\Documents\\GitHub\\TXO_TX_Strategy\\Data\\OptionOpen'